In [1]:
import jax.numpy as jnp
from jax import random

from sgmcmcjax.samplers import build_sgld_sampler


def loglikelihood(theta, x):
    return -0.5 * jnp.dot(x - theta, x - theta)


def logprior(theta):
    return -0.5 * jnp.dot(theta, theta) * 0.01


# generate dataset
N, D = 10_000, 100
key = random.PRNGKey(0)
mu_true = random.normal(key, (D,))
X_data = random.normal(key, shape=(N, D)) + mu_true

# build sampler
batch_size = int(0.1 * N)
dt = 1e-5
sampler = build_sgld_sampler(dt, loglikelihood, logprior, (X_data,), batch_size)

# run sampler
Nsamples = 10_000
samples = sampler(key, Nsamples, jnp.zeros(D))

# test
print(samples.shape)
mu_est = jnp.mean(samples, axis=0)
print(mu_est.shape)
assert jnp.allclose(mu_est, mu_true, atol=1e-1)
print("test passed")

/mnt/lustre/shared_conda/envs/sgreen/PyCBCandOptuna/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
Running for 10,000 iterations: 100%|██████████| 10000/10000 [00:01<00:00, 8900.03it/s]


(10000, 100)
(100,)
test passed
